In [5]:
import torch
import torch.nn as nn
import numpy as np

# Datos de ejemplo (temperaturas en °C)
celsius = np.linspace(-100, 100, 10000).astype(np.float32)
np.savetxt("celsius.txt", celsius, fmt="%.5f")
fahrenheit = celsius * 9/5 + 32
kelvin = celsius + 273.15

# Convertir a tensores
X = torch.tensor(celsius).unsqueeze(1)  # shape (N,1)
Y = torch.tensor(np.stack([fahrenheit, kelvin], axis=1))  # shape (N,2)

# Modelo lineal simple
class TempLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 2)
        )

    def forward(self, x):
        return self.fc(x)

model = TempLinear()

# Entrenamiento rápido
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = nn.MSELoss()

for epoch in range(5000):
    optimizer.zero_grad()
    out = model(X)
    loss = criterion(out, Y)
    loss.backward()
    optimizer.step()





In [2]:
# Prueba
test_c = torch.tensor([[25.0]])
pred = model(test_c)
print(f"{test_c.item()}°C -> {pred.detach().numpy()[0][0]:.2f}°F, {pred.detach().numpy()[0][1]:.2f}K")
traced_model = torch.jit.trace(model, test_c)
traced_model.save("temp_linear.pt")  # este PT sí funciona con RKNN

25.0°C -> 77.00°F, 295.57K


In [3]:
dummy_input = torch.tensor([[0.0]])
torch.onnx.export(
    model,
    dummy_input,
    "temp_linear.onnx",
    export_params=True,
    input_names=["input"],
    output_names=["output"],
)
print("Modelo exportado a temp_linear.onnx")

Modelo exportado a temp_linear.onnx


In [ ]:
from rknn.api import RKNN
import numpy as np

# 1. Prepara configuraciones
rknn = RKNN(verbose=True)

rknn.config(
    mean_values=[0],
    std_values=[1],
    target_platform="rk3588",
)

# 2. Carga tu modelo ONNX
ret = rknn.load_onnx("temp_linear.onnx")
if ret != 0:
    raise RuntimeError("Error cargando el modelo ONNX")

# 3. Build en FP32 (sin cuantizar, sin fusionar)
ret = rknn.build(do_quantization=False)
if ret != 0:
    raise RuntimeError("Error al construir el modelo RKNN")

# 4. Exporta el modelo
ret = rknn.export_rknn("temp_linear_1.rknn")
if ret != 0:
    raise RuntimeError("Error al exportar el modelo RKNN")

I rknn-toolkit2 version: 2.3.2


D Loading model temp_linear.onnx ...
D Dump temp_linear_enc.rknn ...


I Loading : 100%|██████████████████████████████████████████████████| 2/2 [00:00<00:00, 10551.71it/s]
D base_optimize ...
D base_optimize done.
D 
D fold_constant ...
D fold_constant done.
D 
D correct_ops ...
D correct_ops done.
D 
D fuse_ops ...
D fuse_ops results:
D     convert_gemm_by_exmatmul: remove node = ['/fc/fc.0/Gemm'], add node = ['input_tp', 'input_tp_rs', '/fc/fc.0/Gemm#1', 'output_mm_tp', 'output_mm_tp_rs']
D     unsqueeze_to_4d_transpose: remove node = [], add node = ['input_rs', 'input_tp-rs']
D     convert_exmatmul_to_mul: remove node = ['/fc/fc.0/Gemm#1'], add node = ['/fc/fc.0/Gemm#2']
D     fuse_reshape_transpose: remove node = ['input_tp']
D     bypass_two_reshape: remove node = ['input_tp_rs', 'input_tp-rs']
D     fuse_transpose_reshape: remove node = ['output_mm_tp']
D     fold_constant ...
D     fold_constant done.
D fuse_ops done.
D 
D sparse_weight ...
D sparse_weight done.
D 
I rknn building ...
E RKNN: [23:55:45.220] Unkown op target: 0
I rknn building done.

I RKNN: [23:55:45.218] compress = 0, conv_eltwise_activation_fuse = 1, global_fuse = 1, multi-core-model-mode = 7, output_optimize = 1, layout_match = 1, enable_argb_group = 0, op_group_sram_opt = 0, enable_flash_attention = 0, op_group_nbuf_opt = 0, safe_fuse = 0
I RKNN: librknnc version: 2.3.2 (e045de294f@2025-04-07T19:48:25)
D RKNN: [23:55:45.218] RKNN is invoked
D RKNN: [23:55:45.219] >>>>>> start: rknn::RKNNExtractCustomOpAttrs
D RKNN: [23:55:45.219] <<<<<<<< end: rknn::RKNNExtractCustomOpAttrs
D RKNN: [23:55:45.219] >>>>>> start: rknn::RKNNSetOpTargetPass
D RKNN: [23:55:45.219] <<<<<<<< end: rknn::RKNNSetOpTargetPass
D RKNN: [23:55:45.219] >>>>>> start: rknn::RKNNBindNorm
D RKNN: [23:55:45.219] <<<<<<<< end: rknn::RKNNBindNorm
D RKNN: [23:55:45.219] >>>>>> start: rknn::RKNNEliminateQATDataConvert
D RKNN: [23:55:45.219] <<<<<<<< end: rknn::RKNNEliminateQATDataConvert
D RKNN: [23:55:45.219] >>>>>> start: rknn::RKNNConvStrideFixPass
D RKNN: [23:55:45.219] <<<<<<<< end: rknn::RKNNCon